In [1]:
import pandas as pd

# Load the CSV
df = pd.read_csv("final_urls.csv")

# Split into chunks of 1000
chunk_size = 1000

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    chunk.to_csv(f"urls_part_{i//chunk_size + 1}.csv", index=False)

print("Done! Files created.")

Done! Files created.


In [1]:
import pandas as pd
import glob

# Step 1: Find all CSV files starting with 'properties_part_'
csv_files = glob.glob("properties_part_*.csv")

# Step 2: Read and combine them
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Step 3: Save the combined file
combined_df.to_csv("properties.csv", index=False)

print(f"✅ Combined {len(csv_files)} files into 'properties.csv' successfully!")


✅ Combined 8 files into 'properties.csv' successfully!


In [2]:
import pandas as pd
import glob

# Step 1: Find all CSV files starting with 'properties_part_'
csv_files = glob.glob("properties_part_*.csv")

# Step 2: Read and combine them, keeping 'None' as string
combined_df = pd.concat([pd.read_csv(file, keep_default_na=False) for file in csv_files], ignore_index=True)

# Step 3: Save the combined file
combined_df.to_csv("properties.csv", index=False)

print(f"✅ Combined {len(csv_files)} files into 'properties.csv' successfully! 'None' values are preserved.")

✅ Combined 8 files into 'properties.csv' successfully! 'None' values are preserved.
